In [3]:
import re
import os
import pickle as pkl

slice_ = 25000

In [4]:
runs = [12, 13, 14]
methods = ['random', 'state', 'trace']
bug_types = ['Large LogIndex: %d']
slice_ = 5900
for run in runs:
    for method in methods:
        print(f'{method} run: {run}')
        basedir = f'experiments_/raft_3_3/{method}/output{run}/'
        with open(os.path.join(basedir, f'saved/{method}_stats.pkl'), 'rb') as f:
            stats = pkl.load(f)
        print(f'Length: {len(stats["coverage"])}')
        print(f'Coverage: {stats["coverage"][:slice_][-1]}')
        print(f'Len bug_iters: {len(stats["bug_iterations"])}')
        print(f'Bug_iters:')
        print(stats["bug_iterations"])
        print('-------------------------')


random run: 12
Length: 26081
Coverage: 4023
Len bug_iters: 313
Bug_iters:
[64, 128, 192, 216, 221, 460, 585, 1074, 1235, 1394, 1612, 1720, 1927, 2004, 2052, 2119, 2191, 2272, 2277, 2288, 2304, 2481, 2497, 2613, 2720, 2759, 2769, 2889, 2893, 2906, 2923, 3045, 3181, 3202, 3243, 3540, 3613, 3802, 3837, 3841, 3842, 3861, 3906, 3992, 4241, 4611, 4650, 4816, 4936, 4979, 5079, 5168, 5181, 5591, 5616, 5634, 5681, 5726, 5779, 5959, 6184, 6382, 6449, 6458, 6571, 6646, 6988, 7013, 7185, 7196, 7418, 7428, 7635, 7696, 7705, 7747, 7806, 7842, 8029, 8117, 8121, 8163, 8439, 8454, 8576, 8772, 8827, 8863, 8934, 8977, 9026, 9060, 9240, 9318, 9357, 9386, 9402, 9739, 9757, 9777, 9834, 9942, 10029, 10048, 10200, 10203, 10226, 10350, 10435, 10481, 10567, 10608, 10648, 10750, 10777, 10922, 10955, 11026, 11081, 11246, 11373, 11390, 11437, 11452, 11485, 11643, 11781, 11878, 11961, 12020, 12177, 12187, 12243, 12259, 12262, 12288, 12380, 12381, 12542, 12557, 12568, 12635, 12691, 12708, 12723, 12745, 12777, 12888,

In [7]:
all_bugs = {}
unexpected_bugs = {}
#r'\b\d+\b'
slice_ = 4000
for run in runs:
    for method in methods:
        bugs = []
        if method not in all_bugs.keys():
            all_bugs[method] = []
        if method not in unexpected_bugs.keys():
            unexpected_bugs[method] = []
        # print(f'{method} run: {run}')
        basedir = f'experiments_/raft_3_3/{method}/output{run}/errors/'
        for file in os.listdir(basedir):
            iter = [int(n) for n in re.findall(r'\d+', file)][0]
            with open(f'{basedir}{file}/stderr.log', 'r') as f:
                lines = f.readlines()
            if 'Large LogIndex:' in lines[0]:
                li = [int(n) for n in re.findall(r'\b\d+\b', lines[0])]
                if li[0] < 4:
                    continue
                all_bugs[method].append((run, iter, li[0]))
            elif 'Negative LogIndex!' in lines[0]:
                print(f'Negative LogIndex at {(method, run, iter)}')
                all_bugs[method].append((run, iter, -1))
            else:
                unexpected_bugs[method].append((method, run, iter))
print(all_bugs)
print(unexpected_bugs)



Negative LogIndex at ('state', 13, 8015)
{'random': [(12, 5634, 4), (12, 64, 5), (12, 15689, 4), (12, 12243, 4), (12, 5959, 4), (12, 2052, 4), (13, 1712, 4), (13, 22208, 4), (13, 3854, 5), (13, 12433, 4), (13, 2890, 4), (13, 2837, 4), (13, 23476, 4), (13, 22308, 4), (13, 8063, 4), (13, 22205, 4), (13, 1248, 4), (13, 6204, 4), (14, 1573, 4), (14, 13540, 4), (14, 24414, 4), (14, 24470, 4), (14, 13828, 4), (14, 14854, 4), (14, 21512, 4), (14, 9574, 4), (14, 24108, 5), (14, 6860, 4), (14, 17268, 4), (14, 11100, 4)], 'state': [(12, 26328, 4), (12, 4628, 4), (12, 5874, 4), (12, 11780, 4), (13, 13639, 4), (13, 22344, 4), (13, 3366, 4), (13, 12960, 4), (13, 2994, 4), (13, 7667, 4), (13, 8015, -1), (13, 25920, 5), (13, 16707, 4), (14, 2980, 4), (14, 21399, 4), (14, 16178, 4), (14, 3882, 5), (14, 22305, 4), (14, 1106, 4), (14, 11766, 4), (14, 19643, 4), (14, 25480, 4), (14, 806, 4)], 'trace': [(12, 5505, 4), (12, 19521, 4), (12, 7871, 4), (12, 852, 4), (12, 8520, 4), (13, 22400, 4), (13, 274, 4)